In [13]:
# ライブラリのインポート
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.preprocessing import KBinsDiscretizer
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:

# データの読み込み
df = pd.read_csv("Social_Vulnerability_Index_2018_-_United_States__tract_20250119.csv")

# データの概要を確認
print("データの列名:")
print(df.columns)

print("\n'RPL_THEMES'の概要前処理前:")
print(df["RPL_THEMES"].describe())


データの列名:
Index(['ST', 'STATE', 'ST_ABBR', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
       'E_TOTPOP', 'M_TOTPOP', 'E_HU',
       ...
       'F_NOVEH', 'F_GROUPQ', 'F_THEME4', 'F_TOTAL', 'E_UNINSUR', 'M_UNINSUR',
       'EP_UNINSUR', 'MP_UNINSUR', 'E_DAYPOP', 'Shape'],
      dtype='object', length=124)

'RPL_THEMES'の概要前処理前:
count   72837.000000000000000
mean       -8.611693868500899
std        94.996396802680721
min      -999.000000000000000
25%         0.243100000000000
50%         0.495400000000000
75%         0.747700000000000
max         1.000000000000000
Name: RPL_THEMES, dtype: float64


In [15]:
# 'RPL_THEMES'が0から1の範囲内にあるデータを保持し、コピー
df1 = df[df["RPL_THEMES"].between(0, 1)].copy()

print("\n'RPL_THEMES'の概要（フィルタ後）:")
print(df1["RPL_THEMES"].describe())


'RPL_THEMES'の概要（フィルタ後）:
count   72173.000000000000000
mean        0.499993816247073
std         0.288681229660190
min         0.000000000000000
25%         0.250000000000000
50%         0.500000000000000
75%         0.750000000000000
max         1.000000000000000
Name: RPL_THEMES, dtype: float64


In [16]:
# 'RPL_THEMES' を3クラスに分割（低、中、高） using tertiles
df1['RPL_THEMES_BIN'] = pd.qcut(df1['RPL_THEMES'], q=3, labels=[0, 1, 2])

# 新しいクラスの分布を確認
print("\n'RPL_THEMES_BIN'の分布:")
print(df1['RPL_THEMES_BIN'].value_counts())



'RPL_THEMES_BIN'の分布:
RPL_THEMES_BIN
0    24060
2    24057
1    24056
Name: count, dtype: int64


In [18]:
# 目的変数と説明変数の設定
target_variable = 'RPL_THEMES_BIN'
feature_variables = ['EP_POV', 'EP_UNEMP', 'EP_NOHSDP', 'EP_MINRTY', 'EP_AGE65']
X = df1[feature_variables]
y = df1[target_variable].astype(int)  # カテゴリ型から整数型に変換


# データの先頭を確認
print("\n特徴量の先頭:")
print(X.head())

# データの標準化
X = (X - X.mean()) / X.std()

print("\n目的変数の先頭:")
print(y.head())

# 欠損値の確認
print("\n欠損値の数:")
print(X.isnull().sum())

# 欠損値を中央値で補完
X = X.fillna(X.median())

# 再度欠損値を確認
print("\n欠損値の数（補完後）:")
print(X.isnull().sum())


特徴量の先頭:
                EP_POV          EP_UNEMP          EP_NOHSDP  \
24   8.500000000000000 8.900000000000000  2.000000000000000   
107  7.800000000000000 5.600000000000000 11.699999999999999   
198  8.000000000000000 2.600000000000000  8.699999999999999   
211 11.400000000000000 4.700000000000000  1.300000000000000   
233 17.899999999999999 2.100000000000000  8.800000000000001   

             EP_MINRTY           EP_AGE65  
24  20.300000000000001 11.199999999999999  
107 33.899999999999999 22.100000000000001  
198  1.700000000000000 21.000000000000000  
211  3.500000000000000 20.600000000000001  
233  4.200000000000000 29.699999999999999  

目的変数の先頭:
24     1
107    2
198    1
211    0
233    1
Name: RPL_THEMES_BIN, dtype: int64

欠損値の数:
EP_POV       0
EP_UNEMP     0
EP_NOHSDP    0
EP_MINRTY    0
EP_AGE65     0
dtype: int64

欠損値の数（補完後）:
EP_POV       0
EP_UNEMP     0
EP_NOHSDP    0
EP_MINRTY    0
EP_AGE65     0
dtype: int64


In [37]:
import pandas as pd

# 小数点第8位まで表示する設定
pd.options.display.float_format = '{:0.15f}'.format

# そのままコードを実行
kbd = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
X_binned = kbd.fit_transform(X)
X_binned = pd.DataFrame(X_binned, columns=feature_variables)

print("\n連続変数を3ビンに分割したデータの先頭:")
print(X_binned.head())


連続変数を3ビンに分割したデータの先頭:
             EP_POV          EP_UNEMP         EP_NOHSDP         EP_MINRTY  \
0 1.000000000000000 2.000000000000000 0.000000000000000 1.000000000000000   
1 0.000000000000000 1.000000000000000 1.000000000000000 1.000000000000000   
2 0.000000000000000 0.000000000000000 1.000000000000000 0.000000000000000   
3 1.000000000000000 1.000000000000000 0.000000000000000 0.000000000000000   
4 2.000000000000000 0.000000000000000 1.000000000000000 0.000000000000000   

           EP_AGE65  
0 0.000000000000000  
1 2.000000000000000  
2 2.000000000000000  
3 2.000000000000000  
4 2.000000000000000  


In [38]:
# カイ二乗検定の実施
chi2_stat, p_values = chi2(X_binned, y)

# 統計量とp値をデータフレームにまとめる
chi2_results = pd.DataFrame({
    'Feature': feature_variables,
    'Chi2 Statistic': chi2_stat,
    'p-value': p_values
})

# 統計量でソートしてトップ5を取得
chi2_top5 = chi2_results.sort_values(by='Chi2 Statistic', ascending=False)

print("\nカイ二乗検定による特徴量重要度（トップ5）:")
print(chi2_top5[['Feature', 'Chi2 Statistic', 'p-value']])


カイ二乗検定による特徴量重要度（トップ5）:
     Feature        Chi2 Statistic           p-value
2  EP_NOHSDP 26418.683296464521845 0.000000000000000
0     EP_POV 26048.013400624549831 0.000000000000000
3  EP_MINRTY 13713.022703648970491 0.000000000000000
1   EP_UNEMP 13316.504565824296151 0.000000000000000
4   EP_AGE65  2287.042858585005888 0.000000000000000


In [39]:
# スピアマン相関の計算
spearman_results = []
for feature in feature_variables:
    corr, p = spearmanr(X[feature], y)
    spearman_results.append({
        'Feature': feature,
        'Spearman Correlation': corr,  # 符号付き
        'p-value': round(p, 8)
    })

spearman_df = pd.DataFrame(spearman_results)

# 絶対値でソートしてトップ5を取得
spearman_top5 = spearman_df.reindex(
    spearman_df['Spearman Correlation'].abs().sort_values(ascending=False).index
).head(5)

print("\nスピアマン相関による特徴量重要度（トップ5）:")
print(spearman_top5[['Feature', 'Spearman Correlation', 'p-value']])


スピアマン相関による特徴量重要度（トップ5）:
     Feature  Spearman Correlation           p-value
2  EP_NOHSDP     0.777560089369926 0.000000000000000
0     EP_POV     0.769867580747304 0.000000000000000
1   EP_UNEMP     0.561995184353346 0.000000000000000
3  EP_MINRTY     0.546526127253726 0.000000000000000
4   EP_AGE65    -0.201356601681077 0.000000000000000


In [40]:
### Kendalls Tauによる特徴量重要度の計算
from scipy.stats import kendalltau

kendall_results = []

for feature in feature_variables:

    corr, p = kendalltau(X[feature], y)
    kendall_results.append({
        'Feature': feature,
        'Kendall Tau': corr,
        'p-value': round(p, 8)
    })

kendall_df = pd.DataFrame(kendall_results)

# 絶対値でソートしてトップ5を取得
kendall_top5 = kendall_df.reindex(
    kendall_df['Kendall Tau'].abs().sort_values(ascending=False).index
)

print("\nKendall's Tauによる特徴量重要度（トップ7）:")
print(kendall_top5[['Feature', 'Kendall Tau', 'p-value']])



Kendall's Tauによる特徴量重要度（トップ7）:
     Feature        Kendall Tau           p-value
2  EP_NOHSDP  0.638781154620952 0.000000000000000
0     EP_POV  0.632952621889376 0.000000000000000
1   EP_UNEMP  0.446652595041508 0.000000000000000
3  EP_MINRTY  0.426741083447876 0.000000000000000
4   EP_AGE65 -0.155548319323142 0.000000000000000


In [41]:
# Random Forest モデルの構築
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

# 特徴量重要度の取得
rf_importances = pd.Series(rf.feature_importances_, index=feature_variables)
rf_top5 = rf_importances.sort_values(ascending=False)

print("\nRandom Forest による特徴量重要度（トップ9）:")
print(rf_top5)


Random Forest による特徴量重要度（トップ9）:
EP_POV      0.298084056150380
EP_NOHSDP   0.295770274360944
EP_MINRTY   0.176786814801612
EP_UNEMP    0.128568353794457
EP_AGE65    0.100790500892607
dtype: float64


In [42]:
from xgboost import XGBClassifier

# XGBoostモデルの構築と訓練
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X, y)

# 特徴量重要度の取得
xgb_importances = pd.Series(xgb.feature_importances_, index=feature_variables)
xgb_top5 = xgb_importances.sort_values(ascending=False)

print("\nXGBoost による特徴量重要度（トップ9）:")
print(xgb_top5)


/Users/hiro/.pyenv/versions/3.9.16/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:15:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



XGBoost による特徴量重要度（トップ9）:
EP_POV      0.463402390480042
EP_NOHSDP   0.352279692888260
EP_MINRTY   0.090007804334164
EP_UNEMP    0.059058733284473
EP_AGE65    0.035251379013062
dtype: float32


In [45]:
# SVMのモデル構築
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# パイプラインの作成
svc = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=42))
svc.fit(X, y)

# 重みの取得
svc_weights = pd.Series(svc.named_steps['svc'].coef_[0], index=feature_variables)
svc_top5 = svc_weights.abs().sort_values(ascending=False)

print("\nSVM による特徴量重要度（トップ9）:")
print(svc_top5)


SVM による特徴量重要度（トップ9）:
EP_NOHSDP   2.156098304974876
EP_POV      1.815424765759417
EP_UNEMP    0.635572181303814
EP_MINRTY   0.597628749806063
EP_AGE65    0.180090928876780
dtype: float64


In [46]:
# Lasso回帰のモデル構築
from sklearn.linear_model import LogisticRegression

# パイプラインの作成
lasso = make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='liblinear', random_state=42))
lasso.fit(X, y)

# 重みの取得
lasso_weights = pd.Series(lasso.named_steps['logisticregression'].coef_[0], index=feature_variables)
lasso_top5 = lasso_weights.abs().sort_values(ascending=False)
print("\nLasso回帰による特徴量重要度（トップ9）:")
print(lasso_top5)


Lasso回帰による特徴量重要度（トップ9）:
EP_NOHSDP   3.115316685555746
EP_POV      2.406547358206410
EP_UNEMP    0.919728992228639
EP_MINRTY   0.867641134813764
EP_AGE65    0.284622655895218
dtype: float64


In [47]:
# Naive Bayesのモデル構築
from sklearn.naive_bayes import GaussianNB

# パイプラインの作成
nb = make_pipeline(StandardScaler(), GaussianNB())

# モデルの訓練
nb.fit(X, y)

# 重みの取得
nb_weights = pd.Series(nb.named_steps['gaussiannb'].theta_[0], index=feature_variables)
nb_top5 = nb_weights.abs().sort_values(ascending=False)

print("\nNaive Bayes による特徴量重要度（トップ9）:")
print(nb_top5)


Naive Bayes による特徴量重要度（トップ9）:
EP_NOHSDP   0.753779592307598
EP_POV      0.745852869003636
EP_MINRTY   0.588756756022015
EP_UNEMP    0.550160000837189
EP_AGE65    0.173679567560513
dtype: float64


In [48]:
# 各手法のトップ5をランキング順に取得
top_features_rf = rf_importances.sort_values(ascending=False).index.tolist()
top_features_xgb = xgb_importances.sort_values(ascending=False).index.tolist()
top_features_svc = svc_weights.abs().sort_values(ascending=False).index.tolist()
top_features_lasso = lasso_weights.abs().sort_values(ascending=False).index.tolist()
top_features_nb = nb_weights.abs().sort_values(ascending=False).index.tolist()

# ランキング1から5までのリストを作成
rank = list(range(1, 6))

# 統合データフレームの作成
df_rank = pd.DataFrame({
    'Rank': rank,
    'Random Forest': top_features_rf,
    'XGBoost': top_features_xgb,
    'Lasso': top_features_lasso,
    'SVM': top_features_svc,
    'Naive Bayes': top_features_nb
})

print("\n各手法による特徴量ランキング:")
print(df_rank)


各手法による特徴量ランキング:
   Rank Random Forest    XGBoost      Lasso        SVM Naive Bayes
0     1        EP_POV     EP_POV  EP_NOHSDP  EP_NOHSDP   EP_NOHSDP
1     2     EP_NOHSDP  EP_NOHSDP     EP_POV     EP_POV      EP_POV
2     3     EP_MINRTY  EP_MINRTY   EP_UNEMP   EP_UNEMP   EP_MINRTY
3     4      EP_UNEMP   EP_UNEMP  EP_MINRTY  EP_MINRTY    EP_UNEMP
4     5      EP_AGE65   EP_AGE65   EP_AGE65   EP_AGE65    EP_AGE65


In [49]:
# 各手法のトップ5をランキング順に取得
top_features_rf = rf_importances.sort_values(ascending=False).index.tolist()
top_features_chi2 = chi2_top5['Feature'].tolist()
top_features_spearman = spearman_top5['Feature'].tolist()

# ランキング1から5までのリストを作成
rank = [1, 2, 3, 4, 5]

# 各ランキングに対応する特徴量を取得
features_rf_ranked = rf_importances.sort_values(ascending=False).index.tolist()
features_chi2_ranked = chi2_top5['Feature'].tolist()
features_spearman_ranked = spearman_top5['Feature'].tolist()

# 統合データフレームの作成

rank_table = pd.DataFrame({
    'Rank': rank,
    'Random Forest Importance': features_rf_ranked,
    'Chi-Squared Statistic': features_chi2_ranked,
    'Spearman Correlation': features_spearman_ranked
})

# インデックスをRankに設定
rank_table = rank_table.set_index('Rank')

print("\n各手法のトップ5特徴量を統合した結果:")
print(rank_table)


各手法のトップ5特徴量を統合した結果:
     Random Forest Importance Chi-Squared Statistic Spearman Correlation
Rank                                                                    
1                      EP_POV             EP_NOHSDP            EP_NOHSDP
2                   EP_NOHSDP                EP_POV               EP_POV
3                   EP_MINRTY             EP_MINRTY             EP_UNEMP
4                    EP_UNEMP              EP_UNEMP            EP_MINRTY
5                    EP_AGE65              EP_AGE65             EP_AGE65


項目
* EP_POV → 貧困率
* EP_NOHSDP → 高卒未満の割合
* EP_UNEMP → 失業率
* EP_MINRTY → 少数民族(白人、非ヒスパニック系を除くすべての人)
* EP_AGE65 → 65歳以上の割合

* RPL_THEMES → その地域の脆弱性を%で表す

In [50]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIFの計算関数
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values(by='VIF', ascending=False)

# 最初のVIF計算
vif_data = calculate_vif(X)
print("\n初回のVIF:")
print(vif_data)


初回のVIF:
     feature               VIF
0     EP_POV 2.024807641178533
2  EP_NOHSDP 1.915279483833580
3  EP_MINRTY 1.840969557493327
1   EP_UNEMP 1.613642996203697
4   EP_AGE65 1.209273701564790


### 全ての説明変数がVIF < 5 なので、削除する必要はない。